**This problem set is due Wednesday, September 17, 2025 at 11:59 pm. Please plan ahead and submit your work on time.**

### Problem Set 01: Uninformed Search

In this Problem Set you will implement Breath First Search and Depth First Search and use them to solve the [8 Puzzle Problem](https://en.wikipedia.org/wiki/15_puzzle).

0. [Credit for Contributors (required)](#contributors)
1. [State Representation in the 8 Puzzle Problem (35 points)](#state_representation)
    1. [Successor Function (25 points)](#state_expansion)
    2. [Completing the `PuzzleProblem` class (10 points)](#puzzle_problem_class)
2. [Simple Search (65 points)](#simple_search)
    1. [Breath First Search (35 points)](#bfs_implementation)
    2. [Depth First Search (25 points)](#dfs_implementation)
    3. [BFS vs DFS (5 points)](#bfs_vs_dfs)

**100 points** total for Problem Set 1

## <a name="contributors"></a> Credit for Contributors

List the various students, lecture notes, or online resouces that helped you complete this problem set:

Ex: I worked with Bob on the cat activity planning problem.

<div class="alert alert-info">
Write your answer in the cell below this one.
</div>

Looked at lecture notes and asked chatGPT for help with understanding concepts.

Import the modules needed for this exercise (make sure you execute the cell below by clicking on it and pressing Shift-Enter)

**Do not import any other modules**

In [1]:
%load_ext autoreload
%autoreload 2
from search_classes import SearchNode, Path
from principles_of_autonomy.grader import Grader
from principles_of_autonomy.notebook_tests.pset_1 import TestPSet1

## <a name="state_representation"></a>Problem 1: State Representation in the 8 Puzzle Problem

The puzzle consists of a 3x3 grid with 8 numbered tiles and a missing tile. The objective is to slide the tiles around until all the numbered tiles are ordered and the missing tile stays at the lower right cell of the grid.

<img src="puzzle8.png"/>

To make things simple, we are giving you a possible state representation for the 8-puzzle problem.

We'll represent a given state of the puzzle by a tuple of three internal tuples. Each internal tuple represents a row of the puzzle. The missing tile is represented by $0$.

For example, the puzzle state below:

<img src="example_state.png"/>

is represented by `((1, 2, 3), (8, 0, 4), (7, 6, 5))`.

Below, we are giving you some code to print a puzzle state:


In [2]:
def print_state(state):
    print("+"+ "-"*5+"+")
    for l in state:
        print("|"+ " ".join([str(el) if el!=0 else " " for el in l]) +"|")
    print("+"+ "-"*5+"+")

example_state = ((1, 2, 3), (8, 0, 4), (7, 6, 5))

print("%s state represents puzzle state: " % (example_state,))
print_state(example_state)

((1, 2, 3), (8, 0, 4), (7, 6, 5)) state represents puzzle state: 
+-----+
|1 2 3|
|8   4|
|7 6 5|
+-----+


### <a name="state_expansion"></a>1.A Successor Function (25 points)

In order to find a solution to the search problem, we need to define the states we can reach from a given state. This corresponds to the possible moves of the missing tile (at most up, down, left and right).

Implement the function `expand_state(state)` that returns a `list` of the states that can be reached from the given `state`.

For example, for state `((0, 1, 3), (4, 2, 5), (7, 8, 6))`, the function `expand_state` should return the following list (two moves are feasible):

```
[((4, 1, 3), (0, 2, 5), (7, 8, 6)), ((1, 0, 3), (4, 2, 5), (7, 8, 6))]
```

The neighbour states of state:

```
+-----+
|  1 3|
|4 2 5|
|7 8 6|
+-----+
```

are:

```
+-----+
|4 1 3|
|  2 5|
|7 8 6|
+-----+
***
+-----+
|1   3|
|4 2 5|
|7 8 6|
+-----+
```

<div class="alert alert-info">
Implement the function `expand_state(state)` below.
</div>


In [3]:
# 1.A — successor function
def expand_state(state):
    br = bc = None
    for r in range(3):
        for c in range(3):
            if state[r][c] == 0:
                br, bc = r, c
                break
        if br is not None:
            break

    moves = [(-1, 0), (1, 0), (0, -1), (0, 1)]
    succ = []

    for dr, dc in moves:
        nr, nc = br + dr, bc + dc
        if 0 <= nr < 3 and 0 <= nc < 3:
            grid = [list(row) for row in state]
            grid[br][bc], grid[nr][nc] = grid[nr][nc], grid[br][bc]
            succ.append(tuple(tuple(row) for row in grid))

    return succ


In [4]:
Grader.run_single_test_inline(TestPSet1, "test_1_check_expanded_states", locals())

.
----------------------------------------------------------------------
Ran 1 test in 0.002s

OK


### <a name="puzzle_problem_class"></a> 1.B Completing the `PuzzleProblem` class (10 points)

We are giving you the class `SearchNode` defined in `search_classes.py`. This class represents a search node in the search tree. You can create a `SearchNode` by giving it the state it represents and its `SearchNode` parent (or None if it's the root element in the tree). Below is an example of the `Search Node` class being used:

In [5]:
# Execute this example code
root_node = SearchNode(((0, 1, 3), (4, 2, 5), (7, 8, 6)), parent_node=None)
children_node = SearchNode(((4, 1, 3), (0, 2, 5), (7, 8, 6)),
                            parent_node=root_node)
print("Root node: %s" % root_node)
print("Children node: %s" % children_node)

Root node: <SearchNode: state: ((0, 1, 3), (4, 2, 5), (7, 8, 6)), parent: None>
Children node: <SearchNode: state: ((4, 1, 3), (0, 2, 5), (7, 8, 6)), parent: <SearchNode: state: ((0, 1, 3), (4, 2, 5), (7, 8, 6)), parent: None>>


We also give you the `Path` class, that takes a `SearchNode` and computes the state path from the initial state in the root of the tree to the state of the given `SearchNode`:

In [6]:
# Execute this example code
example_path = Path(children_node)
print("Path of %d states is: %s" % (len(example_path.path), example_path.path))

Path of 2 states is: [((0, 1, 3), (4, 2, 5), (7, 8, 6)), ((4, 1, 3), (0, 2, 5), (7, 8, 6))]


Implement the function `expand_node(self, search_node)` inside the `PuzzleProblem` class. The function should return a `list` of the successor SearchNodes that can be reached from the given `search_node`.


<div class="alert alert-warning">
You will want to look at the `SearchNode` and `Path` definitions in the included `search_classes.py` file, as you will need to know what useful properties you can use for the next questions.
</div>

<div class="alert alert-info">
Implement the function `expand_node(self, search_node)` below.
</div>

In [7]:
class PuzzleProblem(object):
    """Class that represents the puzzle search problem."""
    def __init__(self, start, goal):
        self.start = start
        self.goal = goal
    def test_goal(self, state):
        return self.goal == state
    def expand_node(self, search_node):
        """
        Return a list of SearchNodes for the successors of `search_node`.
        Each returned SearchNode must have `search_node` as its parent.
        """
        children = []
        for child_state in expand_state(search_node.state):
            children.append(SearchNode(child_state, parent_node=search_node))
        return children

In [8]:
Grader.run_single_test_inline(TestPSet1, "test_2_puzzle_problem_expanded_nodes", locals())

.
----------------------------------------------------------------------
Ran 1 test in 0.001s

OK


## <a name="simple_search"></a>Problem 2: Simple Search

Now you will implement Simple Search, as seen in class, to solve the 8 Puzzle Problem. 

### <a name="bfs_implementation"></a>2.A Breadth First Search (35 points)

First, you'll implement *Breath First Search*.

Implement the function `breadth_first_search(search_problem)` that takes an instance of the `PuzzleProblem` class that we defined above and returns a tuple of three elements, in the following order:

1. If BFS finds a solution, an instance of the `Path` class containing that solution. If it doesn't, it should return `None` as the first element of the tuple.
2. The number of visited nodes
3. The maximum size of the queue

You should use a **visited list**, as otherwise the number of explored states in this problem will be large.

<div class="alert alert-info">
Implement `breadth_first_search(search_problem)` below.
</div>

In [9]:
def breadth_first_search(search_problem):

    start_node = SearchNode(search_problem.start, parent_node=None)

    if search_problem.test_goal(start_node.state):
        return (Path(start_node), 1, 1)

    queue = [start_node]
    visited = {start_node.state}
    num_visited = 0
    max_q = 1

    while queue:
        max_q = max(max_q, len(queue))
        node = queue.pop(0)
        num_visited += 1

        for child in search_problem.expand_node(node):
            if child.state in visited:
                continue
            if search_problem.test_goal(child.state):
                return (Path(child), num_visited + 0, max(max_q, len(queue)))
            visited.add(child.state)
            queue.append(child)

    return (None, num_visited, max_q)


### Solve the Puzzle Problem using BFS

Let's use your Breath First Search implementation to solve the 8 Puzzle Problem.
Execute the cell below. If your BFS implementation is correct, you should see the solution printed below.

In [10]:
# Solve the 8 Puzzle Problem from state:
# +-----+
# |  1 3|
# |4 2 5|
# |7 8 6|
# +-----+
# Don't modify this cell (contents will be overwritten by autograder)
# If you want to experiment with other states, try adding cells below.
# You can try with state: ((1, 8, 2), (0, 4, 3), (7, 6, 5)) for example.
# Remember that not all states have a solution. Try ((8, 1, 2), (0, 4, 3), (7, 6, 5)), for example.
# Be ready to wait, though!
start_state = ((0, 1, 3), (4, 2, 5), (7, 8, 6))
# start_state = ((1, 8, 2), (0, 4, 3), (7, 6, 5))
goal_state = ((1,2,3),(4,5,6),(7,8,0))
problem = PuzzleProblem(start_state, goal_state)

sol, num_visited, max_q = breadth_first_search(problem)
if sol:    
    print(
        "Solution found!\n%d states in the solution (%d moves)\n"
        "%d states explored.\n%d maximum queue"
        % (len(sol.path), len(sol.path) - 1, num_visited, max_q)
    )
    print("Solution: ")
    for s in sol.path:
        print_state(s)
        print("\n**\n")
else:
    print("No solution after exploring %d states with max q of %d" %(num_visited, max_q))

Solution found!
5 states in the solution (4 moves)
14 states explored.
14 maximum queue
Solution: 
+-----+
|  1 3|
|4 2 5|
|7 8 6|
+-----+

**

+-----+
|1   3|
|4 2 5|
|7 8 6|
+-----+

**

+-----+
|1 2 3|
|4   5|
|7 8 6|
+-----+

**

+-----+
|1 2 3|
|4 5  |
|7 8 6|
+-----+

**

+-----+
|1 2 3|
|4 5 6|
|7 8  |
+-----+

**



In [11]:
Grader.run_single_test_inline(TestPSet1, "test_3_bfs", locals())

.
----------------------------------------------------------------------
Ran 1 test in 0.002s

OK


Solution: 
+-----+
|  1 3|
|4 2 5|
|7 8 6|
+-----+

**

+-----+
|1   3|
|4 2 5|
|7 8 6|
+-----+

**

+-----+
|1 2 3|
|4   5|
|7 8 6|
+-----+

**

+-----+
|1 2 3|
|4 5  |
|7 8 6|
+-----+

**

+-----+
|1 2 3|
|4 5 6|
|7 8  |
+-----+

**



### <a name="dfs_implementation"></a>2.B Depth First Search (25 points)

Next, you'll implement *Depth First Search*.

Implement the function `depth_first_search(search_problem)` that takes an instance of the `PuzzleProblem` class that we defined above and also returns a tuple of three elements, in the following order:

1. If DFS finds a solution, an instance of the `Path` class containing that solution. If it doesn't, it should return `None` as the first element of the tuple.
2. The number of visited nodes
3. The maximum size of the queue

You should use a **visited list**, as otherwise the number of explored states in this problem will be large.

<div class="alert alert-info">
Implement `depth_first_search(search_problem)` below.
</div>

In [12]:
def depth_first_search(search_problem):
    root = SearchNode(search_problem.start)
    if search_problem.test_goal(root.state):
        return (Path(root), 1, 1)

    stack = [root]
    visited = set()
    num_visited = 0
    max_q = len(stack)

    while stack:
        node = stack.pop()
        if node.state in visited:
            continue
        visited.add(node.state)
        num_visited += 1
        if search_problem.test_goal(node.state):
            return (Path(node), num_visited, max_q)

        children = search_problem.expand_node(node)
        for child in children:
            if child.state not in visited:
                stack.append(child)
        if len(stack) > max_q:
            max_q = len(stack)

    return (None, num_visited, max_q)

### Solve the Puzzle Problem using DFS

Let's use your Depth First Search implementation to solve the 8 Puzzle Problem.
Execute the cell below. If your DFS implementation is correct, you should find a very long solution.

In [13]:
# Solve the 8 Puzzle Problem from state:
# +-----+
# |  1 3|
# |4 2 5|
# |7 8 6|
# +-----+
# Don't modify this cell (contents will be overwritten by autograder)
# If you want to experiment with other states, try adding cells below.
# You can try with state: ((1, 8, 2), (0, 4, 3), (7, 6, 5)) for example.
# Remember that not all states have a solution. Try ((8, 1, 2), (0, 4, 3), (7, 6, 5)), for example.
# Be ready to wait, though!
start_state = ((0, 1, 3), (4, 2, 5), (7, 8, 6))
# start_state = ((1, 8, 2), (0, 4, 3), (7, 6, 5))
goal_state = ((1,2,3),(4,5,6),(7,8,0))
problem = PuzzleProblem(start_state, goal_state)

sol, num_visited, max_q = depth_first_search(problem)
if sol:
    print("Solution found!\n%d states in the solution (%d moves)\n%d states explored.\n%d maximum queue"
          % (len(sol.path), len(sol.path)-1, num_visited,max_q)
          )
else:
    print("No solution after exploring %d states with max q of %d" %(num_visited, max_q))

Solution found!
24157 states in the solution (24156 moves)
24773 states explored.
17837 maximum queue


In [14]:
# Note: the test case uses an instructor implementation of expand_nodes
Grader.run_single_test_inline(TestPSet1, "test_4_dfs", locals())

.
----------------------------------------------------------------------
Ran 1 test in 1.338s

OK


Solution: 
+-----+
|1 2 3|
|4 5 6|
|7   8|
+-----+

**

+-----+
|1 2 3|
|4 5 6|
|7 8  |
+-----+

**



### <a name="bfs_vs_dfs"></a> 2.C BFS vs DFS (5 points)

Would DFS be a better choice for this problem? What benefits does BFS have over DFS in this problem? Please explain in the cell below (double click on the cell below, remove the text and type your answer). This is a qualitative question, you are not required to give numerical comparisons.

DFS is not suitable for this problem, because for DFS, once the algorithm goes into the wrong path, it takes very long time to get out of that detour, resulting in very long solutions. Instead, BFS made sure that the simple solutions are fully explored before moving on to longer solutions. For this problem, BFS results in shorter solution, less search operations and faster runtime.

### <a name="Time Spent"></a> 3. Time Spent on Pset (5 points)

Please use [this form](https://forms.gle/LRVH2WwatrjakJGJA) to tell us how long you spent on this pset. After you submit the form, the form will give you a confirmation word. Please enter that confirmation word below to get an extra 5 points. 

In [15]:
form_confirmation_word = "Apple pie"

In [16]:
# Run all tests
Grader.grade_output([TestPSet1], [locals()], "results.json")
Grader.print_test_results("results.json")

Total score is 100/100.

Score for test_1_check_expanded_states (principles_of_autonomy.notebook_tests.pset_1.TestPSet1) is 25/25.

Score for test_2_puzzle_problem_expanded_nodes (principles_of_autonomy.notebook_tests.pset_1.TestPSet1) is 10/10.

Score for test_3_bfs (principles_of_autonomy.notebook_tests.pset_1.TestPSet1) is 35/35.
- Solution: 
+-----+
|  1 3|
|4 2 5|
|7 8 6|
+-----+

**

+-----+
|1   3|
|4 2 5|
|7 8 6|
+-----+

**

+-----+
|1 2 3|
|4   5|
|7 8 6|
+-----+

**

+-----+
|1 2 3|
|4 5  |
|7 8 6|
+-----+

**

+-----+
|1 2 3|
|4 5 6|
|7 8  |
+-----+

**



Score for test_4_dfs (principles_of_autonomy.notebook_tests.pset_1.TestPSet1) is 25/25.
- Solution: 
+-----+
|1 2 3|
|4 5 6|
|7   8|
+-----+

**

+-----+
|1 2 3|
|4 5 6|
|7 8  |
+-----+

**



Score for test_5_form_word (principles_of_autonomy.notebook_tests.pset_1.TestPSet1) is 5/5.
